# LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록, `Runnable` 프로토콜을 구현했다.

`Runnable` 프로토콜은 대부분 컴포넌트에 구현되어 있다.

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith("LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-Advanced


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [10]:
model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=512
)

template = PromptTemplate.from_template("{tech}에 대하여 3문장으로 설명해줘")

chain = template | model | StrOutputParser()

## stream : 실시간 출력
이 함수는 스트림을 반복하여 각 데이터의 내용(context)을 즉시 출력한다.

end=""인자는 출력 후 줄바꿈을 하지 않도록 설정하며, flush=True 인자는 출력 버퍼를 즉시 비우도록 한다.

In [11]:
for token in chain.stream("비행기"):
    print(token,end="",flush=True)

비행기는 공중에서 사람이나 화물을 운송하기 위해 설계된 항공기입니다. 일반적으로 날개와 엔진을 갖추고 있으며, 공기역학 원리를 이용해 비행합니다. 현대의 비행기는 상업용 여객기부터 군사기, 화물기 등 다양한 종류가 있습니다.

## invoke : 호출
invoke 메서드는 주제를 인자로 받아 해당 주자에 대한 처리를 수행한다.

In [12]:
chain.invoke("비행기")

'비행기는 공중을 비행할 수 있도록 설계된 교통 수단으로, 주로 사람이나 화물을 운송하는 데 사용됩니다. 항공기의 비행 원리는 양력, 중력, 추진력, 항력의 상호 작용에 기반하며, 이를 통해 하늘을 날 수 있습니다. 현대의 비행기는 다양한 기술과 안전 장치를 갖추고 있어 효율적이고 안전한 항공 여행을 제공합니다.'

# batch : 배치(단위 실행)
- 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 디셔너리에 있는 topic 키의 값을 사용하여 일괄 처리

In [14]:
answer = chain.batch([{"tech":"컴퓨터"},{"tech":"머그컵"}])

In [15]:
answer[0]

'컴퓨터는 데이터를 처리하고 저장하는 전자 기기로, 다양한 작업을 수행할 수 있는 프로그램을 실행할 수 있습니다. 일반적으로 하드웨어와 소프트웨어로 구성되어 있으며, 하드웨어는 물리적인 부품, 소프트웨어는 이들 부품을 제어하는 프로그램을 포함합니다. 컴퓨터는 정보 검색, 계산, 통신 등 여러 분야에서 중요한 역할을 하고 있습니다.'

In [16]:
answer[1]

'머그컵은 일반적으로 손잡이가 있는 깊고 넓은 컵으로, 주로 뜨거운 음료를 마시는 데 사용됩니다. 다양한 디자인과 재질로 제작되어 개인의 취향에 맞게 선택할 수 있으며, 커피, 차, 핫초코 등 다양한 음료를 담기에 적합합니다. 또한, 머그컵은 종종 선물용으로도 인기가 있으며, 개인화된 메시지나 이미지로 장식되기도 합니다.'

max_concurrency 매개변수를 사용하여 동시 요청 수를 설정할 수 있다.

config 딕셔너리는 max_concurrency 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정한다.

In [18]:
answer = chain.batch(
    [
        {"tech": "ChatGPT"},
        {"tech": "Instagram"},
        {"tech": "멀티모달"},
        {"tech": "프로그래밍"},
        {"tech": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

In [19]:
answer

['ChatGPT는 OpenAI에서 개발한 인공지능 언어 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 자연스럽고 유창한 텍스트로 응답할 수 있도록 훈련되었습니다. 다양한 주제에 대해 대화할 수 있으며, 정보 제공, 문제 해결, 창의적인 글쓰기 등의 용도로 활용됩니다.',
 'Instagram은 사용자가 사진과 동영상을 공유하고 소통할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자들이 창의적으로 콘텐츠를 제작할 수 있도록 돕습니다. 또한, 스토리, IGTV, 리얼스와 같은 기능을 통해 실시간으로 소통하고 다양한 형식의 콘텐츠를 소비할 수 있는 환경을 제공합니다.',
 '멀티모달(Multimodal)이라는 개념은 다양한 형태의 데이터 또는 정보(예: 텍스트, 이미지, 오디오 등)를 결합하여 처리하거나 분석하는 방식을 의미합니다. 이러한 접근은 서로 다른 모드 간의 상호작용을 통해 더 풍부하고 정확한 이해를 가능하게 합니다. 최근 인공지능 분야에서는 멀티모달 학습이 자연어 처리와 컴퓨터 비전 기술을 통합하여 더욱 효과적인 모델을 개발하는 데 활용되고 있습니다.',
 '프로그래밍은 컴퓨터가 특정 작업을 수행하도록 지시하는 과정으로, 주로 프로그래밍 언어를 사용하여 코드로 작성됩니다. 이 과정에서는 알고리즘을 설계하고, 문제를 해결하기 위한 로직을 구현하며, 결과를 검증하는 단계를 포함합니다. 프로그래밍은 소프트웨어 개발, 데이터 분석, 웹 개발 등 다양한 분야에서 활용되며, 현대 사회에서 필수적인 기술로 자리 잡고 있습니다.',
 '머신러닝은 데이터에서 패턴을 학습하고 예측을 수행하는 인공지능의 한 분야입니다. 알고리즘을 사용하여 주어진 데이터를 분석하고, 이를 통해 새로운 데이터에 대한 결정을 자동으로 내릴 수 있도록 합니다. 머신러닝은 이미지 인식, 자연어 처리, 추천 시스템 등 다양한 분야에서 활용되고 있습니다.']

# async stream : 비동기 스트림

chain.astream 은 비동기 스트림을 생성하며, 주어진 토픽에 대한 정보를 비동기적으로 처리한다.

비동기 for루프(async for)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, print 함수를 통해 메시지의 내용(content)를 즉시 출력한다.

In [20]:
async for token in chain.astream({"tech":"나무망치"}):
    print(token,end="",flush=True)

나무망치는 주로 나무로 만들어진 도구로, 주로 못을 박거나 물체를 부드럽게 두드리는 데 사용됩니다. 금속망치에 비해 충격이 덜해 손상 위험이 낮아 섬세한 작업에 적합합니다. 또한, 나무망치는 가벼운 무게 덕분에 사용하기 쉽고 다양한 작업에 활용될 수 있습니다.

# async invoke : 비동기 호출

chain 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행한다.

In [21]:
async_process = chain.ainvoke({"tech":"맥북"})

In [22]:
await async_process

'맥북은 애플이 개발한 노트북 컴퓨터로, macOS 운영 체제를 사용합니다. 고급스러운 디자인과 뛰어난 성능을 갖추고 있으며, 사용자 친화적인 인터페이스로 많은 사람들에게 사랑받고 있습니다. 또한, 애플의 생태계와 잘 통합되어 있어 아이폰, 아이패드 등과의 연동이 용이합니다.'

# async batch : 비동기 배치

`abatch` 비동기적으로 일련의 작업을 일괄 처리

In [23]:
async_batch = chain.abatch(
    [
        {"tech": "ChatGPT"},
        {"tech": "Instagram"},
    ],
)

In [24]:
await async_batch

['ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 자연어 처리를 통해 사용자와의 대화를 지원합니다. 이 모델은 방대한 양의 텍스트 데이터를 학습하여 다양한 주제에 대해 질문에 답하거나 정보를 제공할 수 있습니다. 사용자는 ChatGPT를 통해 정보 검색, 글쓰기 보조, 그리고 다양한 주제에 대한 토론 등을 할 수 있습니다.',
 '인스타그램은 사용자들이 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자가 창의적으로 콘텐츠를 꾸밀 수 있도록 돕습니다. 또한, 친구들과의 소통뿐만 아니라 브랜드와의 연결을 통해 마케팅 및 비즈니스 기회도 제공합니다.']

---

# Parallel : 병렬성

In [32]:
from langchain_core.runnables import RunnableParallel

# {tech} 사용법
chain1 = (
    PromptTemplate.from_template("{tech} 의 사용법은 뭐야? 1문장으로")
    | model 
    | StrOutputParser()
)

# {tech} 구매처
chain2 = (
    PromptTemplate.from_template("{tech} 의 구매처는 어디야? 1문장으로")
    | model 
    | StrOutputParser()
)

combined = RunnableParallel(use=chain1,buy=chain2)

In [26]:
combined.invoke({"tech":"고무줄"})

{'use': '고무줄은 다양한 용도로 사용되는 유용한 도구입니다. 여기 몇 가지 일반적인 사용법을 소개합니다:\n\n1. **물건 묶기**: 고무줄은 여러 물건을 함께 묶는 데 유용합니다. 예를 들어, 종이를 묶거나 여러 개의 연필을 함께 고정할 때 사용합니다.\n\n2. **서류 정리**: 서류나 문서를 정리할 때 고무줄을 사용하여 묶어두면 쉽게 보관할 수 있습니다.\n\n3. **임시 고정**: 어떤 물체를 일시적으로 고정할 필요가 있을 때, 고무줄을 사용하여 간편하게 고정할 수 있습니다. 예를 들어, 먼지나 이물질이 들어가지 않도록 물건을 덮을 때 사용합니다.\n\n4. **DIY와 공예**: 고무줄은 공예 프로젝트나 DIY 작업에서도 자주 사용됩니다. 예를 들어, 고무줄을 이용해 다양한 형태를 만들거나 색상을 조합할 수 있습니다.\n\n5. **운동 용품**: 고무줄은 운동 용품으로도 활용됩니다. 저항 밴드로 사용하여 근력 운동을 할 때 사용되기도 합니다.\n\n6. **음식 포장**: 식품 포장에도 사용됩니다. 예를 들어, 남은 음식을 포장할 때 고무줄을 이용해 밀폐할 수 있습니다.\n\n이처럼 고무줄은 일상생활에서 매우 다양한 용도로 사용될 수 있는 유용한 아이템입니다.',
 'buy': '고무줄은 다양한 장소에서 구매할 수 있습니다. 아래는 몇 가지 구매처입니다:\n\n1. **문구점**: 일반적인 문구점에서는 다양한 크기의 고무줄을 판매합니다.\n2. **온라인 쇼핑몰**: 쿠팡, G마켓, 11번가 등 다양한 온라인 쇼핑몰에서 고무줄을 쉽게 구매할 수 있습니다.\n3. **대형 마트**: 이마트, 롯데마트와 같은 대형 마트에서도 고무줄을 찾아볼 수 있습니다.\n4. **사무용품 전문점**: 사무용품을 전문으로 하는 매장에서도 고무줄을 판매합니다.\n\n필요한 크기나 종류에 따라 적절한 구매처를 선택하면 좋습니다!'}

In [29]:
combined.invoke({"tech1":"볼펜","tech2":"샤프심"})

{'use': '볼펜의 사용법은 매우 간단합니다. 아래의 단계를 따라 사용해 보세요:\n\n1. **볼펜 준비**: 볼펜의 캡을 제거하거나, 버튼식이라면 버튼을 눌러 펜심을 앞으로 내립니다.\n\n2. **쓰기**: 종이나 원하는 표면에 볼펜을 대고 글씨를 씁니다. 볼펜은 일반적으로 아래에서 위로, 왼쪽에서 오른쪽으로 쓰는 것이 일반적입니다.\n\n3. **종이 선택**: 볼펜은 다양한 종류의 종이에 사용할 수 있지만, 일반적으로 평평하고 부드러운 표면에서 잘 작동합니다.\n\n4. **보관**: 사용 후에는 캡을 다시 닫거나, 버튼을 눌러 펜심을 내부로 집어넣어 잉크가 마르지 않도록 합니다.\n\n5. **교체**: 잉크가 다 소진된 경우, 볼펜의 리필 잉크를 구입하여 교체할 수 있습니다. 일부 볼펜은 리필이 가능하고, 일부는 일회용입니다.\n\n볼펜은 간편하게 사용할 수 있어 일상적인 메모, 필기 등에 널리 사용됩니다.',
 'buy': '샤프심은 다양한 문구점이나 온라인 쇼핑몰에서 구매할 수 있습니다. 한국에서는 다음과 같은 곳에서 샤프심을 찾을 수 있습니다:\n\n1. **문구점**: 가까운 문구점이나 대형 문구 전문 매장에서 구매할 수 있습니다.\n2. **대형 마트**: 이마트, 홈플러스 같은 대형 마트에서도 문구류 코너에서 판매합니다.\n3. **온라인 쇼핑몰**: 쿠팡, 11번가, G마켓, 옥션 등 다양한 온라인 플랫폼에서 샤프심을 구매할 수 있습니다.\n4. **전문 문구점**: 알파문구, 문구사랑 같은 전문 문구점에서도 여러 종류의 샤프심을 취급합니다.\n\n원하는 브랜드나 타입에 따라 적절한 곳에서 구매하시면 됩니다.'}

# 배치에서의 병렬 처리

In [31]:
chain1.batch(([{"tech":"에어팟"},{"tech":"후추"}]))

['에어팟(AirPods)의 사용법은 다음과 같습니다:\n\n### 1. 연결하기\n- **첫 연결:** 에어팟을 케이스에서 꺼내면 자동으로 전원이 켜집니다. 아이폰이나 iPad의 블루투스를 켜고, 에어팟을 가까이 두면 화면에 연결 팝업이 나타납니다. "연결" 버튼을 누르면 연결됩니다.\n- **다른 기기와 연결:** 다른 블루투스 기기와 연결하려면 에어팟의 뒷면에 있는 버튼을 눌러 페어링 모드로 전환한 후, 해당 기기에서 블루투스를 켜고 에어팟을 선택합니다.\n\n### 2. 착용\n- 에어팟을 귀에 꽂고 착용합니다. 두 개의 에어팟 모두 착용하면 스테레오 사운드를 들을 수 있습니다.\n\n### 3. 제어 방법\n- **재생/일시 정지:** 에어팟의 한쪽을 두 번 탭하면 음악이 재생되거나 일시 정지됩니다.\n- **다음 트랙:** 오른쪽 에어팟을 두 번 탭합니다.\n- **이전 트랙:** 왼쪽 에어팟을 두 번 탭합니다.\n- **Siri 호출:** 에어팟의 한쪽을 길게 누르면 Siri가 활성화됩니다.\n\n### 4. 배터리 확인\n- 에어팟의 배터리 상태는 아이폰이나 아이패드에서 에어팟을 가까이 두면 팝업으로 확인할 수 있습니다. 또한, 에어팟 케이스의 LED 표시등을 통해 배터리 상태를 확인할 수 있습니다.\n\n### 5. 충전하기\n- 에어팟은 충전 케이스에 넣어 충전합니다. 케이스는 Lightning 케이블로 충전할 수 있습니다.\n\n### 6. 설정 조정\n- 아이폰의 설정에서 "Bluetooth"를 선택한 후, 에어팟 옆의 "i" 아이콘을 눌러 다양한 설정(예: 자동 귀 감지, 이름 변경 등)을 조정할 수 있습니다.\n\n이 외에도 여러 기능이 있으니, 사용하면서 다양한 기능을 활용해 보세요!',
 '후추는 다양한 요리에 사용되는 향신료로, 주로 맛을 더하고 향을 향상시키는 데 사용됩니다. 후추의 사용법은 다음과 같습니다:\n\n1. **조리 시 사용**: 후추는 고기, 생선, 채소 등 다양한 재료에 조리 중에 넣어 풍미를 더할 수 있습니다.

In [33]:
chain2.batch(([{"tech":"에어팟"},{"tech":"후추"}]))

['에어팟은 애플 공식 웹사이트, 애플 스토어, 전자제품 매장 및 온라인 쇼핑몰에서 구매할 수 있습니다.',
 '후추는 슈퍼마켓, 온라인 쇼핑몰, 그리고 재래시장에서 구매할 수 있습니다.']

In [34]:
combined.batch(([{"tech":"에어팟"},{"tech":"후추"}]))

[{'use': '에어팟은 Bluetooth 기능이 있는 기기와 연결하여 음악을 듣거나 전화를 받을 수 있으며, 이어폰을 귀에 꽂으면 자동으로 연결되고, 터치 또는 제스처로 다양한 기능을 조작할 수 있습니다.',
  'buy': '에어팟은 애플 공식 웹사이트, 애플 스토어, 전자제품 매장 및 온라인 쇼핑몰에서 구매할 수 있습니다.'},
 {'use': '후추는 음식에 풍미를 더하고, 소화를 촉진하며, 향신료로 다양한 요리의 맛을 개선하는 데 사용됩니다.',
  'buy': '후추는 대형 마트, 온라인 쇼핑몰, 또는 지역 식료품점에서 구매할 수 있습니다.'}]